In [ ]:
import pandas as pd
import pickle
import numpy as np

In [ ]:
predictions_over_time_path = '/Users/jk1/temp/opsum_prediction_output/LSTM_72h_testing/3M_mRS02/2023_01_02_1057/test_LSTM_sigmoid_all_balanced_0.2_2_True_RMSprop_3M mRS 0-2_16_3/predictions_over_timesteps.pkl'
features_path = '/Users/jk1/temp/opsum_prepro_output/gsu_prepro_01012023_233050/preprocessed_features_01012023_233050.csv'
labels_path = '/Users/jk1/temp/opsum_prepro_output/gsu_prepro_01012023_233050/preprocessed_outcomes_01012023_233050.csv'

In [ ]:
outcome =  "3M mRS 0-2"
test_size = 0.2
seed = 42
n_time_steps = 72

In [ ]:
with open(predictions_over_time_path, 'rb') as handle:
    predictions_over_time = pickle.load(handle)

In [ ]:
from sklearn.model_selection import train_test_split
from prediction.outcome_prediction.data_loading.data_formatting import format_to_2d_table_with_time, \
    link_patient_id_to_outcome, features_to_numpy

# load the dataset
X, y = format_to_2d_table_with_time(feature_df_path=features_path, outcome_df_path=labels_path,
                                    outcome=outcome)

# Reduce every patient to a single outcome (to avoid duplicates)
all_pids_with_outcome = link_patient_id_to_outcome(y, outcome)
pid_train, pid_test, y_pid_train, y_pid_test = train_test_split(all_pids_with_outcome.patient_id.tolist(),
                                                                all_pids_with_outcome.outcome.tolist(),
                                                                stratify=all_pids_with_outcome.outcome.tolist(),
                                                                test_size=test_size,
                                                                random_state=seed)

test_X_df = X[X.patient_id.isin(pid_test)]
test_y_df = y[y.patient_id.isin(pid_test)]

test_X_np = features_to_numpy(test_X_df,
                              ['case_admission_id', 'relative_sample_date_hourly_cat', 'sample_label', 'value'])
test_y_np = np.array([test_y_df[test_y_df.case_admission_id == cid].outcome.values[0] for cid in
                      test_X_np[:, 0, 0, 0]]).astype('float32')

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_scores = []
for ts in range(n_time_steps):
    modified_time_steps = ts + 1
    y_pred = predictions_over_time[ts]
    roc_auc_scores.append([modified_time_steps, roc_auc_score(test_y_np, y_pred)])

In [ ]:
results_df = pd.DataFrame(roc_auc_scores, columns=['n_hours', 'roc_auc_score'])
results_df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import os

ax = sns.lineplot(x='n_hours', y='roc_auc_score', data=results_df, legend=True)
ax.set_title('Model performance in the holdout test dataset as a function of observation period')
ax.set_xlabel('Time after admission (hours)')
ax.set_ylabel('ROC AUC')
ax.set_ybound(0.8,0.9)

plt.tight_layout()
# plt.show()

In [ ]:
plt.savefig(os.path.join('/Users/jk1/Downloads', f'{outcome}_roc_auc_scores_over_time.png'), bbox_inches='tight')

## Performance at 24h

In [ ]:
from sklearn.utils import resample

roc_auc_scores_bs = []
ts = 24

n_iterations = 1000
for i in range(n_iterations):
    y_pred_bs, y_bs = resample(predictions_over_time[ts], test_y_np, replace=True)

    # evaluate model
    roc_auc_bs = roc_auc_score(y_bs, y_pred_bs)
    roc_auc_scores_bs.append(roc_auc_bs)

median_roc_auc = np.percentile(roc_auc_scores_bs, 50)
# get 95% interval
alpha = 100 - 95
lower_ci_roc_auc = np.percentile(roc_auc_scores_bs, alpha / 2)
upper_ci_roc_auc = np.percentile(roc_auc_scores_bs, 100 - alpha / 2)

print(median_roc_auc, lower_ci_roc_auc, upper_ci_roc_auc)